<a href="https://colab.research.google.com/github/spike-h/mdai/blob/main/notebooks/MDai_Infographics_Placer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report to Infographics using templates


Below is a script to generate an infographics script from a medical report, using mdai's GPT4 api, and then place this script onto an infographics template. The locations and color of the texts have to be predefined. Accompanying images will also be generated, and their locations also have to be predefined. The script will maximize the size of the text to fit in the bounding box.

______

  Inputs:

    `template` - either a file path or file-like object that can be opened with Image.open()
    `num_sections` - the number of header, text pairs you want to include in the infographic. The lengths of the following header and text lists must be equal to num_sections.
    `api_key` - Pexels API key for image scraping. Leave this as "" if you don't want to insert images. If this is "" then the following image arguments will be ingored.
    `header_positions` & `text_positions` & `img_positions` - a list of the top left positions of their corresponding bounding boxes. Indicates where the text/image will be placed. Format: [(x,y),...] in pixels
    `header_rects` & `text_rects` & `img_bboxes` - a list of bounding boxes for the texts/images to be placed in. Format: [[width,height],...]
    `header_colors` & `text_colors` - a list of rbg colors for the headers and texts. Format: [(r,b,g,a),...]
    `mdai_client` - MD.aiclient instantiated with mdai.Client()
Outputs:

`
  There will be a file called infographic.jpg in your cwd with the placed text and images.
`

## Example Usage

In [ ]:
import mdai
import io

report = """
Age: 42
History: Family history of breast cancer, mother

Examination: Bilateral Digital 2D Screening Mammogram
Technique: Digital 2D CC and MLO views are obtained.

Findings:

Breast Tissue density: A

There is an oval circumscribed mass measuring 8mm in the upper-outer left breast. An oval circumscribed mass of approximate 8mm is also observed in the middle-outer right breast.

Otherwise, there is no suspicious mass, architectural distortion or group microcalcifications.

Impression:
Right and left breast mass for ultrasound imaging recommended.

BI-RADS 0
"""

header_positions = [(430,326), (430,662), (430,1000), (430,1335)]
header_rects = [[688,33] for i in range(4)]
header_colors = [(238,137,153), (255,245,163), (120,190,130), (137,208,226)]
text_positions = [(430,380), (430,721), (430,1046), (430,1386)]
text_rects = [[684,112] for i in range(4)]
text_colors = [(255, 255, 255, 183) for i in range(4)]
img_positions = [(1405, 299), (1400, 629), (1418, 970), (1406,1289)]
img_bboxes = [[209, 206] for i in range(4)]
num_sections = 4

response = requests.get('https://raw.githubusercontent.com/spike-h/mdai/main/misc/temp.png')
template = io.BytesIO(response.content)


# Get variables from project info tab and user settings
# DOMAIN = 'public.md.ai'
# YOUR_PERSONAL_TOKEN = '8b80c4ca0f0587'
# mdai_client = mdai.Client(domain=DOMAIN, access_token=YOUR_PERSONAL_TOKEN)

draw_all_text(template=template, report=report,
              header_positions=header_positions,
              header_rects=header_rects,
              header_colors=header_colors,
              text_positions=text_positions,
              text_rects=text_rects,
              text_colors=text_colors,
              img_positions=img_positions,
              img_bboxes=img_bboxes,
              num_sections=num_sections,
              api_key = 'kjhfkjahiu2o4u5'
              mdai_client=mdai_client)

## Source Code

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from urllib.request import urlopen
import requests
import mdai
import re
import random

# Get gpt inforgaphics script. Requires instantiated mdai client using mdai.Client
def gpt_script(report, mdai_client, num_sections):
  prompt = f'Generate text for an infographic explaining the given report in a patient friendly way. Give it in {num_sections} sections.'

  messages = [{"role": "user", "content": report + '/n' + prompt}]
  reply = mdai_client.chat_completion.create(messages, model='gpt-4', temperature=0)

  text = reply['choices'][0]['message']['content']
  text = re.sub('[\n]+', '\n', text) # removes double new line breaks
  text = text.split('\n')

  headers = text[::2]
  texts = text[1::2]

  return headers, texts

def place_text(draw, location, rect, font_url, text, color, title=False):
  # find font size for text `"Hello World, "` to fit in rectangle 200x100
  selected_size = 1
  selected_text = text
  for size in range(5, 150):
    fit = True
    arial = ImageFont.FreeTypeFont(urlopen(font_url), size=size)
    left, top, right, bottom = draw.multiline_textbbox((0,0), text, font=arial)
    w = right - left
    h = bottom - top

    words = text.split()
    new_text = ""

    if w > rect[0]:
      for word in words:
        left, top, right, bottom = draw.multiline_textbbox((0,0), new_text + word + " ", font=arial)
        w = right - left
        h = bottom - top

        if w > rect[0]:
          new_text += '\n' + word + ' '
        else:
          new_text += word + ' '

        if h > rect[1]:
          fit = False
          break

    if new_text != "":
        left, top, right, bottom = draw.multiline_textbbox((0,0), new_text, font=arial)
        w = right - left
        h = bottom - top

    if w > rect[0] or h > rect[1]:
      fit = False
      break

    if fit:
      selected_size = size
      selected_text = new_text

  if selected_text == "":
    selected_text = text

  selected_font = ImageFont.FreeTypeFont(urlopen(font_url), size=selected_size)

  offset = [left, top]
  true_loc =  (location[0] - offset[0], location[1] - offset[1])



  if title:
    draw.text(true_loc, selected_text, fill=color, font=selected_font, anchor='mm')
  else:
    draw.text(true_loc, selected_text, fill=color, font=selected_font, anchor='la')

def draw_all_text(*, template, report, header_positions, header_rects, header_colors,
                  text_positions, text_rects, text_colors,
                  img_positions, img_bboxes, num_sections, api_key, mdai_client):
  headers, texts = gpt_script(report, mdai_client, num_sections)

  font_url = 'https://github.com/googlefonts/roboto/blob/main/src/hinted/Roboto-Regular.ttf?raw=true' # not gpt

  # Load image
  image = Image.open(template)
  draw = ImageDraw.Draw(image)

  # Draw headers and texts
  for i in range(len(header_positions)):
    place_text(draw, header_positions[i], header_rects[i], font_url, headers[i], header_colors[i])
    place_text(draw, text_positions[i], text_rects[i], font_url, texts[i], text_colors[i])

  if api_key:
    place_images(image, img_positions, img_bboxes, api_key)

def place_images(original, img_positions, img_bboxes, api_key):
  current = original
  for pos, bbox in zip(img_positions, img_bboxes):
    img = download_random_image(api_key)
    img = img.resize((bbox[0], bbox[1]), Image.ANTIALIAS)

    watermark_layer = Image.new("RGBA", original.size)
    watermark_layer.paste(img, pos)

    watermarked = Image.alpha_composite(current.convert('RGBA'), watermark_layer)
    current = watermarked

  watermarked_rgb = watermarked.convert('RGB')
  watermarked_rgb.save('infographic.jpg', 'JPEG')

def download_random_image(api_key):
  url = "https://pixabay.com/api/"

  params = {
      'key': api_key,
      'q': 'medical',
      'image_type': 'vector',
      'per_page': 200
  }

  response = requests.get(url, params=params)

  if response.status_code == 200:
      data = response.json()
      images = data['hits']
      random_image = random.choice(images)
      image_url = random_image['webformatURL']
      print(image_url)
      response = requests.get(image_url)

      if response.status_code == 200:
          image = Image.open(io.BytesIO(response.content))
          return image
      else:
          print('Unable to download image')
  else:
      print('Unable to fetch images')